# 🛰️ Satellite Image Super-Resolution - Inference Demo

**Purpose**: Run inference on a test image using our trained SwinIR model

**What this notebook does**:
1. Sets up the environment on Google Colab
2. Downloads the pre-trained model checkpoint
3. Loads a test image
4. Performs 4x super-resolution
5. Displays and saves the results

**Runtime**: ~2-5 minutes on Colab (with GPU)

## Step 1: Check GPU Availability

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ Running on CPU (slower). For GPU: Runtime → Change runtime type → T4 GPU")

## Step 2: Clone the Repository

In [ ]:
# Clone the project repository
!git clone https://github.com/Elkrisent/klymo-ascent-1.0.git
%cd klymo-ascent-1.0

## Step 3: Install Dependencies

In [ ]:
!pip install -q torch torchvision pillow numpy matplotlib rasterio scikit-image
print("✅ Dependencies installed")

## Step 4: Download Pre-trained Model

**Note**: The model checkpoint should be available in the repository. If not, you can upload it manually or download from Google Drive.

In [ ]:
import os

# Check if checkpoint exists
checkpoint_path = "checkpoints/swinir_best.pth"
if os.path.exists(checkpoint_path):
    print(f"✅ Model checkpoint found: {checkpoint_path}")
else:
    print(f"⚠️ Checkpoint not found at {checkpoint_path}")
    print("You can:")
    print("  1. Upload swinir_best.pth to the checkpoints/ folder")
    print("  2. Or download from Google Drive (if you have the link)")
    
    # Uncomment and modify if you have a Google Drive link:
    # from google.colab import drive
    # drive.mount('/content/drive')
    # !cp /content/drive/MyDrive/path/to/swinir_best.pth checkpoints/

## Step 5: Prepare Test Image

You can either:
- **Option A**: Use the included test image from the repository
- **Option B**: Upload your own satellite image (GeoTIFF or PNG/JPG)

In [ ]:
# Option A: Use test image from repository (if available)
test_image_path = "data/mystery_location/mystery_sr.tif"

# Option B: Upload your own image
# Uncomment the following lines to upload:
# from google.colab import files
# uploaded = files.upload()
# test_image_path = list(uploaded.keys())[0]

print(f"Test image: {test_image_path}")

## Step 6: Load and Display the Input Image

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

try:
    # Try loading as GeoTIFF
    import rasterio
    from rasterio.plot import reshape_as_image
    
    with rasterio.open(test_image_path) as src:
        img = src.read()
        img = reshape_as_image(img)
        print(f"Loaded GeoTIFF: {img.shape}, dtype: {img.dtype}")
        
        # Normalize to 8-bit if needed
        if img.dtype == np.uint16:
            img = (img.astype(float) / 65535 * 255).astype(np.uint8)
        
        lr_image = img
except:
    # Load as regular image
    lr_image = np.array(Image.open(test_image_path))
    print(f"Loaded image: {lr_image.shape}")

# Ensure RGB format
if lr_image.ndim == 2:
    lr_image = np.stack([lr_image]*3, axis=-1)
elif lr_image.shape[2] > 3:
    lr_image = lr_image[:, :, :3]

print(f"Input image shape: {lr_image.shape}")

# Display input
plt.figure(figsize=(10, 10))
plt.imshow(lr_image)
plt.title("Input Low-Resolution Image")
plt.axis('off')
plt.show()

## Step 7: Load the SwinIR Model

In [ ]:
import torch
import sys
sys.path.append('.')

from models.swinir_satellite import SatelliteSwinIR

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize model
model = SatelliteSwinIR(scale=4).to(device)

# Load checkpoint
checkpoint = torch.load(checkpoint_path, map_location=device)
state_dict = checkpoint.get('model_state_dict', checkpoint)
model.load_state_dict(state_dict)
model.eval()

# Display model info
if 'psnr' in checkpoint:
    print(f"✅ Model loaded successfully! (Validation PSNR: {checkpoint['psnr']:.2f} dB)")
else:
    print("✅ Model loaded successfully!")
    
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

## Step 8: Run Inference

We'll process the image in tiles if it's large to avoid memory issues.

In [ ]:
def inference_single_patch(lr_image, model, device):
    """Run inference on a single patch"""
    # Convert to tensor
    lr_tensor = torch.from_numpy(lr_image).permute(2, 0, 1).float() / 255.0
    lr_tensor = lr_tensor.unsqueeze(0).to(device)
    
    # Run inference
    with torch.no_grad():
        sr_tensor = model(lr_tensor)
    
    # Convert back to numpy
    sr_image = sr_tensor.squeeze(0).cpu().permute(1, 2, 0).numpy()
    sr_image = np.clip(sr_image * 255.0, 0, 255).astype(np.uint8)
    
    return sr_image


def tile_inference(large_lr_image, model, device, tile_size=128, overlap=16):
    """Process large image by tiling with overlap"""
    h, w, c = large_lr_image.shape
    scale = 4
    
    sr_h, sr_w = h * scale, w * scale
    sr_canvas = np.zeros((sr_h, sr_w, c), dtype=np.float32)
    weight_map = np.zeros((sr_h, sr_w), dtype=np.float32)
    
    stride = tile_size - overlap
    print(f"Processing {h}x{w} image in {tile_size}x{tile_size} tiles...")
    
    tile_count = 0
    for y in range(0, h - tile_size + 1, stride):
        for x in range(0, w - tile_size + 1, stride):
            lr_tile = large_lr_image[y:y+tile_size, x:x+tile_size, :]
            sr_tile = inference_single_patch(lr_tile, model, device)
            
            # Calculate SR coordinates
            y_sr, x_sr = y * scale, x * scale
            tile_h_sr, tile_w_sr = tile_size * scale, tile_size * scale
            
            # Create weight for smooth blending
            weight = np.ones((tile_h_sr, tile_w_sr), dtype=np.float32)
            if overlap > 0:
                fade_size = overlap * scale
                fade = np.linspace(0, 1, fade_size, dtype=np.float32)
                weight[:fade_size, :] *= fade[:, np.newaxis]
                weight[-fade_size:, :] *= fade[::-1, np.newaxis]
                weight[:, :fade_size] *= fade[np.newaxis, :]
                weight[:, -fade_size:] *= fade[::-1][np.newaxis, :]
            
            # Accumulate
            sr_canvas[y_sr:y_sr+tile_h_sr, x_sr:x_sr+tile_w_sr, :] += (
                sr_tile.astype(np.float32) * weight[:, :, np.newaxis]
            )
            weight_map[y_sr:y_sr+tile_h_sr, x_sr:x_sr+tile_w_sr] += weight
            
            tile_count += 1
            if tile_count % 10 == 0:
                print(f"  Processed {tile_count} tiles...")
    
    # Normalize by weights
    sr_canvas = sr_canvas / (weight_map[:, :, np.newaxis] + 1e-8)
    sr_canvas = np.clip(sr_canvas, 0, 255).astype(np.uint8)
    
    print(f"✅ Tiling complete ({tile_count} tiles)")
    return sr_canvas


# Decide whether to use tiling
h, w = lr_image.shape[:2]
if h <= 256 and w <= 256:
    # Small image - process directly
    print("Processing image directly (no tiling needed)...")
    sr_image = inference_single_patch(lr_image, model, device)
else:
    # Large image - use tiling
    sr_image = tile_inference(lr_image, model, device, tile_size=128, overlap=16)

print(f"✅ Super-resolution complete!")
print(f"Output shape: {sr_image.shape} (4x upscale)")

## Step 9: Display Results

In [ ]:
# Display side-by-side comparison
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

axes[0].imshow(lr_image)
axes[0].set_title(f"Input LR Image\n{lr_image.shape[0]}×{lr_image.shape[1]}", fontsize=16)
axes[0].axis('off')

axes[1].imshow(sr_image)
axes[1].set_title(f"Output SR Image (4x)\n{sr_image.shape[0]}×{sr_image.shape[1]}", fontsize=16)
axes[1].axis('off')

plt.tight_layout()
plt.show()

## Step 10: Display Detail Comparison

Let's zoom into a specific region to see the quality improvement.

In [ ]:
# Extract a patch for comparison
patch_size = 64
lr_patch = lr_image[:patch_size, :patch_size]
sr_patch = sr_image[:patch_size*4, :patch_size*4]

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(lr_patch)
axes[0].set_title(f"LR Patch ({patch_size}×{patch_size})", fontsize=14)
axes[0].axis('off')

axes[1].imshow(sr_patch)
axes[1].set_title(f"SR Patch ({patch_size*4}×{patch_size*4})", fontsize=14)
axes[1].axis('off')

plt.tight_layout()
plt.show()

## Step 11: Save Results

In [ ]:
# Save SR image
output_path = "sr_output.png"
Image.fromarray(sr_image).save(output_path)
print(f"✅ Saved super-resolved image to: {output_path}")

# Download the result
try:
    from google.colab import files
    files.download(output_path)
    print("✅ File download started")
except:
    print("(Download manually or use Colab files panel)")

## Step 12: Quality Metrics (Optional)

If you have a high-resolution ground truth image, you can compute PSNR and SSIM metrics.

In [ ]:
# Optional: Compute metrics if you have ground truth
# Uncomment if you have an HR ground truth image:

# from skimage.metrics import peak_signal_noise_ratio as psnr
# from skimage.metrics import structural_similarity as ssim
# 
# # Load ground truth
# hr_image = np.array(Image.open("path_to_ground_truth.png"))
# 
# # Compute metrics
# psnr_value = psnr(hr_image, sr_image)
# ssim_value = ssim(hr_image, sr_image, channel_axis=2, data_range=255)
# 
# print(f"PSNR: {psnr_value:.2f} dB")
# print(f"SSIM: {ssim_value:.4f}")

print("✅ Inference demo complete!")

---

## Summary

This notebook demonstrates:
- ✅ Loading a pre-trained SwinIR model
- ✅ Processing satellite images with 4x super-resolution
- ✅ Handling large images with tiled inference
- ✅ Visualizing and saving results

**Model Details**:
- Architecture: SwinIR (Swin Transformer for Image Restoration)
- Scale: 4x upsampling
- Input: Low-resolution satellite imagery (RGB)
- Output: High-resolution satellite imagery (4× larger)

**Use Cases**:
- Enhancing satellite imagery resolution
- Improving detail in Earth observation data
- Supporting analysis tasks requiring higher spatial resolution